# Projeto de Web - Scraping

Projeto desenvolvido como desafio para participação no Intensivão de Python promovido pela #Hastag Treinamentos

A Empresa fictícia para a qual estamos desenvolvendo o projeto comercializa produtos que são diariamente influenciados por moedas estrangeiras (Dólar, Euro) e ouro.

Meu trabalho como analista de vendas é atualizar a cotação das moedas diariamente  para termos novo preço de venda.

Desafio: Atualizar a cotação das moedas diariamente e com esses valores atualizarmos o novo preço de vendas da Planilha de Produtos do cliente.

In [122]:
# Para saber quais bibliotecas já tenho instaladas
#pip freeze

In [123]:
# Instalar Selenium ( Utilizada para automações Web - um robô)
!pip install selenium

In [124]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

navegador = webdriver.Chrome()
navegador.get("https://www.google.com/")

In [125]:
!pip install pandas
import pandas as pd

In [126]:
!pip install openpyxl

## Iniciando o Web - Scraping para pegar as cotações (Dólar,Euro e Ouro)

In [127]:
# Passo 1: Pegar a cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_dolar)



5.053999999999999


In [128]:
# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

5.257


In [129]:
# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)



293.52


### Atualizando a  base de preços do cliente com as novas cotações

In [130]:
# Passo 4: Importar a lista de produtos
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [131]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação com a moeda corrente
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.054,5053.94946,1.40,7075.529244
1,Carro Renault,4500.00,Euro,5.257,23656.50000,2.00,47313.000000
2,Notebook Dell,899.99,Dólar,5.054,4548.54946,1.70,7732.534082
3,IPhone,799.00,Dólar,5.054,4038.14600,1.70,6864.848200
4,Carro Fiat,3000.00,Euro,5.257,15771.00000,1.90,29964.900000
5,Celular Xiaomi,480.48,Dólar,5.054,2428.34592,2.00,4856.691840
6,Joia 20g,20.00,Ouro,293.520,5870.40000,1.15,6750.960000


## Exportar a nova base de preços atualizada

In [132]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Precos Atualizados.xlsx", index=False)